In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
import cv2
import matplotlib.pyplot as plt

import keras.layers as layers
from keras.layers import (Dense, Activation, LayerNormalization,Conv2DTranspose,add, 
    BatchNormalization, Dropout, Input, Flatten, Conv2D, MaxPool2D, Reshape,UpSampling2D)
from keras import Model

In [ ]:
MAIN_ROOT = '../input/sartorius-cell-instance-segmentation'

train_csv_path = os.path.join(MAIN_ROOT, 'train.csv')
train_image_path = os.path.join(MAIN_ROOT, 'train')

In [ ]:
IMAGE_WIDTH = 704
IMAGE_HEIGHT = 520

BATCH_SIZE = 16

In [ ]:
train_data = pd.read_csv(train_csv_path).fillna(-1)
train_data.head()

In [ ]:
#train_data = train_data.drop(['cell_type', 'plate_time', 'sample_date', 'sample_id', 'elapsed_timedelta', 'height', 'width'], axis = 1)
train_data = train_data.drop(['cell_type', 'plate_time', 'sample_date', 'sample_id', 'elapsed_timedelta'], axis = 1)

train_data.head()
train_data.iloc[0]['annotation']

In [ ]:
grouped_EncodedPixels = train_data.groupby('id').apply(list)
grouped_EncodedPixels.head()

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:

class DataGenerator(keras.utils.Sequence):
    
    def __init__(self, list_IDs, labels, image_dir, batch_size=32, img_width=704, img_height=520, n_channels=1,
                 n_classes=1, shuffle=True):
        'Initialization'
        self.img_width = img_width
        self.img_height = img_height
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.image_dir = image_dir
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.labels = labels

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.img_height, self.img_width, self.n_channels))
        y = np.empty((self.batch_size, self.img_height, self.img_width, self.n_classes))

        # Generate data
        for idx, id_ in enumerate(list_IDs_temp):
            
            file_path = os.path.join(self.image_dir, id_ + '.png')
            
            image = cv2.imread(file_path,0) # load gray-scale image
            
            image_resized = np.array(image, dtype=np.float64) 
            
            image_resized -= image_resized.mean()
            image_resized /= image_resized.std()
            
            mask = np.zeros((self.img_height, self.img_width, self.n_classes))
            
            db_annotations = labels[labels['id'] == id_]
            db_annotations = db_annotations.reset_index()
            
            for i in range(0,len(db_annotations)):
                annotation = db_annotations['annotation'][i].split()
                starts, lengths = [np.asarray(x, dtype=int) for x in (annotation[0:][::2], annotation[1:][::2])]
                starts -= 1
                ends = starts + lengths
                img = np.zeros(db_annotations['width'][0]*db_annotations['height'][0], dtype=np.uint8)
                img = np.where(img==0, np.nan, img)
                for lo, hi in zip(starts, ends):
                    img[lo:hi] = 1
                    
                img = img.reshape((self.img_height, self.img_width, self.n_classes))
                mask += np.nan_to_num(img)
            
            # Store sample
            X[idx,] = np.expand_dims(image_resized, axis=2)

            # Store class
            y[idx,] = mask
            
        #y = (y > 0).astype(int)
            
        return X, y
    

In [ ]:
params = {'batch_size':BATCH_SIZE, 
          'img_width':IMAGE_WIDTH, 
          'img_height':IMAGE_HEIGHT, 
          'n_channels':1,
          'n_classes':1, 
          'shuffle':False}

Data_ids = train_data['id'].unique()

labels = train_data[['id', 'annotation', 'width','height']]

val_size = 100
#shuffle data
np.random.shuffle(Data_ids)

#create train and validation subsets
#datagen created in such a way, that labels variable same for train and val
tr_ids = Data_ids[:-val_size]

val_ids = Data_ids[-val_size:]

print(tr_ids.shape)
print(val_ids.shape)
print(train_data.describe())

for id in tr_ids:
    if id in val_ids:
        print('kekw')
        break
        
#create datagens for train and valid datasets
Tr_datagen = DataGenerator(tr_ids, labels,  train_image_path,  **params)
Val_datagen = DataGenerator(val_ids, labels,  train_image_path,  **params)

In [ ]:
DataGen = DataGenerator(Data_ids, labels,  train_image_path,  **params)

In [ ]:
data_sample = DataGen.__getitem__(0)
x, y = data_sample
print(x.shape, y.shape)

In [ ]:
def plot_images(id_,plot_image=True):
    shape=(520, 704)
    db_annotations = train_data[train_data['id'] == id_]
    db_annotations = db_annotations.reset_index()
    cell_photo = Image.open("../input/sartorius-cell-instance-segmentation/train/{}.png".format(id_))
    list_img= []
    for i in range(0,len(db_annotations)):
        annotation = db_annotations['annotation'][i].split()
        starts, lengths = [np.asarray(x, dtype=int) for x in (annotation[0:][::2], annotation[1:][::2])]
        starts -= 1
        ends = starts + lengths
        img = np.zeros(db_annotations['width'][0]*db_annotations['height'][0], dtype=np.uint8)
        img = np.where(img==0, np.nan, img)
        for lo, hi in zip(starts, ends):
            img[lo:hi] = 1
            
        img = img.reshape(shape)
        list_img.append(img)
        
        
        
    print('Size of list: {}'.format(len(list_img)))
    print('Size of mask: {}'.format(list_img[0].shape))

    plt.figure(figsize = (12,9))
    if plot_image:
        plt.imshow(cell_photo,cmap='gray',aspect='auto')
    for i in range(0,len(list_img)):
        plt.imshow(list_img[i])
        
    return list_img

In [ ]:

img, mask = x[0], y[0]
plt.figure(figsize=(50,25))
plt.subplot(2,1,1)
plt.imshow(img)
plt.subplot(2,1,2)
plt.imshow(mask)


In [ ]:
id__ = Data_ids[0]
list_masks = plot_images(id__, False)

In [ ]:
masks = np.zeros((list_masks[0].shape))
for mask in list_masks:
    masks += np.nan_to_num(mask)

In [ ]:
print(masks)
print(masks.shape)
plt.figure(figsize=(12,9))
plt.imshow(masks)

In [ ]:
def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (1,))
    
    #pad inputs height, because dimensions could be multiple of 2^n, where n - depth of U-net
    padded_inputs = layers.ZeroPadding2D(padding=(4,0))(inputs)

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(padded_inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual
    
    #get a shape of tensor
    x_shape = tf.shape(x)
    #slice a tensor, so we get correct output dimensions
    x = tf.slice(x, [0,4,0,0], [x_shape[0],520,704,1])
    
    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model

keras.backend.clear_session()

# Build model
size = (IMAGE_HEIGHT, IMAGE_WIDTH)
n_classes = 1
model = get_model(size, n_classes)
model.summary()

In [ ]:
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=['accuracy',
                                            'MeanSquaredError','AUC'])

# Train the model, doing validation at the end of each epoch.
epochs = 15
model.fit(Tr_datagen, epochs=epochs, validation_data=Val_datagen)

In [ ]:
model.save('cell_segmentation_model1.h5')